In [25]:
import json
import re
import difflib

'''import dataset'''
file = open('test_ingr.json','r')   #make sure this file is in the same directory as the notebook file
data = json.load(file)

Things to address:
1. Plurals and singulars of the same ingredient were counted as different entries. We want to combine them
2. Separate combined ingredients (e.g. "salt &/and pepper")
3. Find certain adjectives in front of ingredients with useful info (e.g. fresh (spices) --> choppable))
4. Trivial stuff that are kind of/not really important, e.g. soya sauce = soy sauce, cilantro = coriander, etc. (also apparantly capsicums = peppers)

In [26]:
'''get ingredient list'''
ingr = set()

for x in data[:2000]:
    for i in range(len(x['ingredients'])):
            if x['valid'][i]:
                ingr.add(x['ingredients'][i]['text'])
ingr = list(ingr) # turns set into list

#ingrSorted = sorted(ingr)

#print(ingrSorted)
#print(len(ingrSorted))

In [27]:
'''count frequency of each ingredient'''

print(len(data))
# print('data[0]:', data[0])

ingr_raw = []  # keeps track of all ingredients in all recipes
for x in data[:2000]:
    for i in range(len(x['ingredients'])):
            if x['valid'][i]:
                ingr_raw.append(x['ingredients'][i]['text'])
                
                
def freq_dict(lst):   
    '''counts occurrences of elements in a list'''
    food_dict = {} # key: ingredient name;  value: count of occurences
    for i in lst:
        if i not in food_dict:
            food_dict[i] = 1
        else:
            food_dict[i] += 1
    return sorted(food_dict.items(), key = lambda kv: kv[1])

print(freq_dict(ingr_raw))

1228
[('dried bay leaf', 1), ('sandwich buns', 1), ('fresh parmesan cheese', 1), ('cheese ravioli', 1), ('red snapper', 1), ('minced beef', 1), ('lemon pepper seasoning', 1), ('pate', 1), ('fresh meyer lemon juice', 1), ('chilies', 1), ('squash', 1), ('citric acid', 1), ('greens', 1), ('pine needles', 1), ('maple bacon', 1), ('unsweetened applesauce', 1), ('dried breadcrumbs', 1), ('sandwich bun', 1), ('fennel', 1), ('bamboo skewers', 1), ('red capsicums', 1), ('skinless boneless turkey breasts', 1), ('microwave popcorn', 1), ('okra', 1), ('aubergines', 1), ('coconut milk powder', 1), ('cherry pie filling', 1), ('lime zest', 1), ('shimeji mushrooms', 1), ('unsweetened pineapple chunks', 1), ('dried red pepper flakes', 1), ('white chocolate baking squares', 1), ('Italian style breadcrumbs', 1), ('dried pasilla peppers', 1), ('MINUTE White Rice', 1), ('prawns', 1), ('hamburger patties', 1), ('filet of beef', 1), ('fresh peaches', 1), ('pink peppercorns', 1), ('yellow food coloring', 1), 

The regex

In [28]:
def match_re(regex, target_lst):
    '''finds elements in target list that matches with the regex'''
    output = []
    for element in target_lst:   
        for match in re.finditer(regex, element):
            # finditer: Returns an iterator yielding match objects over all
            # non-overlapping matches for the RE pattern in string.
            output.append(match.group(0))
    return output

### Dealing with the plurals 

In [29]:
'''the regex'''
all_problems = re.compile(r'''
                        \w+(\s*)\&(\s*)\w+        # A & B 
                        |\w+(\s*)('and')(\s*)\w+  # A and B
                        |(\w+)s\b                 # words that end with s                 
                        ''', re.VERBOSE)

s_endings = re.compile(r'''
                        (\w+)s\b                 # words that end with s                 
                        ''', re.VERBOSE)

In [30]:
'''counts the frequencies of ingredients with words ending with s'''

s_ending_ingr = match_re(s_endings, ingr)

def count_dict(lst):
    output = {}
    for i in lst:
        if i not in output:
            output[i] = 1
        else:
            output[i] += 1
    return sorted(output.items(), key = lambda kv: kv[1])

s_ingr_dict = count_dict(s_ending_ingr)
print(s_ingr_dict)


[('truffles', 1), ('stems', 1), ('Grahams', 1), ('hummus', 1), ('rabbits', 1), ('artichokes', 1), ('chickens', 1), ('skins', 1), ('cakes', 1), ('cornichons', 1), ('haricots', 1), ('fruits', 1), ('chickpeas', 1), ('scallions', 1), ('Less', 1), ('quinces', 1), ('bagels', 1), ('breads', 1), ('habaneros', 1), ('crepes', 1), ('fries', 1), ('glutinous', 1), ('roasts', 1), ('spears', 1), ('Seasonings', 1), ('whites', 1), ('pecans', 1), ('nonpareils', 1), ('currants', 1), ('confectioners', 1), ('aubergines', 1), ('Foods', 1), ('delicious', 1), ('pomegranates', 1), ('bulbs', 1), ('Rolls', 1), ('eggplants', 1), ('wafers', 1), ('pigeons', 1), ('pepitas', 1), ('bratwursts', 1), ('leeks', 1), ('prawns', 1), ('toothpicks', 1), ('cross', 1), ('tangerines', 1), ('melts', 1), ('hunts', 1), ('yams', 1), ('mussels', 1), ('shallots', 1), ('cardamoms', 1), ('crisps', 1), ('franks', 1), ('sheets', 1), ('sausages', 1), ('crusts', 1), ('segments', 1), ('muffins', 1), ('lemongrass', 1), ('pimentos', 1), ('endi

Most ingredients are actually written in plural, so extracting words ending with s probably won't work. Finding the singular might be more practical, except the regex is hard to write and we might as well list them manually.

### Dealing with combined ingredients

In [31]:
'''the regex'''

and_ingr = re.compile(r'''
                        (\w+(\s))+\&((\s)\w+)+        # A & B 
                        |(\w+(\s))+('and')((\s)\w+)+  # A and B             
                        ''', re.VERBOSE)

and_match_ingr = match_re(and_ingr, ingr_raw)
print(and_match_ingr)
print(freq_dict(and_match_ingr))

['salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'thick & chunky salsa', 'salt & pepper', 'salt & freshly ground black pepper', 'salt & fresh ground pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly ground black pepper', 'salt & freshly 

In [64]:

# still having problems making a useful regex that splits the "and" expressions correctly
# also idk if the amount of these ingredients would actually affect our results


print(freq_dict(and_match_ingr))
print()

re_pepper = re.compile(r'(\w+\s)*black pepper(\s\w+)*')   
# can't just use "pepper" because it'd return all kinds of weird-coloured peppers
pepper_match = match_re(re_pepper, ingr_raw)

print(freq_dict(pepper_match))
print()

re_salt = re.compile(r'(\w+\s)*salt(\s\w+)*')
salt_match = match_re(re_salt, ingr_raw)

print(freq_dict(salt_match))



#for i in cp:
#    print(re.compile("(\w\w+(\s))(\w\w+(\s))").split(i))
#    #new_list1.append()
#print(new_list1)


#for i in and_match_ingr:
#    for j in i.split():
#        new_list.append(j)
#print(new_list)
#print()

#for i in new_list:
#    if i =='&':
#        new_list.remove('&')
#        
#print(new_list)    

#----------------------------------------------------------------------------------------------------------------


# Of the 2000 recipes imported in this notebook there are approximately 81 "black pepper" and 
# about 120 "X and xyz black pepper", so for black pepper it might actuallly be important to separate them.
# Same with salt and...?
# For other ingredients though it might be easier just to ignore them since we're just generating recipes with
# more common ingredients.

#[('Philadelphia Chive & Onion 1', 1), ('tomato sauce & mushrooms', 1), ('thick & chunky salsa', 3), 
#('salt & fresh ground pepper', 3), ('salt & pepper', 3), ('salt & freshly ground black pepper', 43)]

[('tomato sauce & mushrooms', 1), ('Philadelphia Chive & Onion 1', 1), ('salt & fresh ground pepper', 3), ('salt & pepper', 3), ('thick & chunky salsa', 3), ('salt & freshly ground black pepper', 43)]

[('coarse black pepper', 1), ('whole black pepper', 2), ('cracked black pepper', 3), ('salt and black pepper', 3), ('fresh coarse ground black pepper', 3), ('ground black pepper', 27), ('freshly ground black pepper', 43), ('fresh ground black pepper', 44), ('black pepper', 81)]

[('dry roasted salt', 1), ('hawaiian sea salt', 1), ('salt pork', 1), ('seasoned salt', 1), ('pickling salt', 1), ('season salt', 1), ('flaked sea salt', 1), ('truffle salt', 1), ('table salt', 2), ('onion salt', 3), ('salt and black pepper', 3), ('fine salt', 5), ('coarse salt', 6), ('celery salt', 6), ('seasoning salt', 7), ('garlic salt', 15), ('sea salt', 18), ('kosher salt', 62), ('salt and pepper', 83), ('salt', 577)]


### Finding  ingredients with useful adjectives 

In [9]:
'''the regex'''
adj_finder = re.compile(r'''
                        fresh(\s\w+)((\s\w+)*)      # insert any adjectives you want        
                        
                        ''', re.VERBOSE)
adj_ingr = match_re(adj_finder, ingr)
print(sorted(adj_ingr))

['fresh Italian parsley', 'fresh apricots', 'fresh asparagus', 'fresh basil', 'fresh basil leaves', 'fresh bay leaves', 'fresh blackberries', 'fresh blueberries', 'fresh breadcrumbs', 'fresh broccoli', 'fresh broccoli florets', 'fresh button mushrooms', 'fresh carrots', 'fresh cherries', 'fresh chervil', 'fresh chives', 'fresh cilantro', 'fresh cilantro leaves', 'fresh cilantro stems', 'fresh coarse ground black pepper', 'fresh coconut', 'fresh coriander', 'fresh coriander leaves', 'fresh corn', 'fresh crabmeat', 'fresh cranberries', 'fresh dill', 'fresh figs', 'fresh flat', 'fresh fruit', 'fresh garlic', 'fresh ginger', 'fresh gingerroot', 'fresh goat cheese', 'fresh goat cheese', 'fresh green beans', 'fresh ground black pepper', 'fresh ground pepper', 'fresh ground pepper', 'fresh ground white pepper', 'fresh herbs', 'fresh jalapeno chilies', 'fresh jalapenos', 'fresh lemon juice', 'fresh lemon rind', 'fresh lemon zest', 'fresh lime juice', 'fresh linguine', 'fresh lump crabmeat', 'f

### Other trivial stuff